This dynamic programing method should keep ans to be smaller than 2048

In [3]:
import numpy as np
import os
import time
time_limit = 30
folder = "Problems2"

In [2]:
def getroute(distances, number, start_time, time_limit):   
    dp = np.ones((number, 2**(number-1)), dtype = np.float16)*60000
    ans = 2048
    is_terminate = False
    
    for j in range(2**(number-1)):
        if time.time() - start_time > time_limit:
            is_terminate = True  
            break
        for i in range(1, number):   
            if j == 0:
                dp[i][j] = distances[i][0] 
            if j>>(i-1)&1 == 1:
                continue
            for k in range(1, number):
                if j>>(k-1)&1 == 0 or i == k:
                    continue
                dp[i][j] = min(dp[i][j], distances[i][k] + dp[k][j ^ (1 << (k-1))])
    if is_terminate:
        
        return 0, []
    ###### get answer
    for i in range(1, number): 
        ans = min(ans, distances[0][i] + dp[i][(2**(number-1)-1) ^ (1 << (i-1))])
    ###### get route
    cur_place = 0
    cur_set = 2**(number-1)-1
    route = [0]
    for step in range(number-2):
        
        cur = 2048
        next_place = 0
        for i in range(1, number):
            if cur_set >> (i-1)&1 == 1:
                if cur > distances[cur_place][i] + dp[i][cur_set ^ (1 << (i-1))]:
                    cur = distances[cur_place][i] + dp[i][cur_set ^ (1 << (i-1))]
                    next_place = i
        cur_set = cur_set ^ (1 << (next_place-1))
        route.append(next_place)
        cur_place = next_place
    route.append(int(np.log2(cur_set))+1)
    route.append(0)
    return ans, "->".join(map(str, route))

In [4]:
result = [[] for _ in os.listdir(folder)]
is_terminate = [False for _ in os.listdir(folder)]
for step, dirs in enumerate(sorted(os.listdir(folder), key = lambda x: int(x))):

    stop = False

    if stop:
        break
    print("Test with number of {} begin!".format(dirs))
    cur_dir = os.path.join(folder, dirs)
    for files in os.listdir(cur_dir):
        if stop:
            break
        start_time = time.time()
        path = os.path.join(cur_dir, files)
        with open(path, 'r', encoding='utf-8') as file:
            content = file.read().split()
            data = list(map(float, content))
            number = int(data[0])
            distances = np.zeros((number,number))
        for i in range(number):
            for j in range(number):
                distances[i][j] = data[i*number+j+1]
        ans, route = getroute(distances, number, start_time, time_limit)
        if ans != 0:
            print("ans is: {}, route is: {}".format(ans, route))
            result[step].append(ans)
        else:
            stop = True
            print("Time Limit Reach!!! No return for {}".format(number))
        
    print()
    print()

Test with number of 16 begin!
ans is: 661.9268840820313, route is: 0->9->8->4->15->10->6->13->14->11->2->1->12->3->5->7->0
ans is: 647.2645705078126, route is: 0->3->15->1->14->7->2->9->10->5->4->11->8->6->12->13->0
ans is: 522.0635893066407, route is: 0->10->15->4->3->8->9->6->1->11->2->7->14->13->5->12->0
ans is: 540.8877485351562, route is: 0->11->9->14->6->10->3->15->7->1->5->4->8->12->13->2->0
ans is: 538.7196740234375, route is: 0->5->6->12->14->7->2->1->13->10->9->4->11->3->8->15->0
ans is: 469.51247490234374, route is: 0->8->9->13->7->4->12->3->1->10->5->15->14->6->2->11->0
ans is: 647.8822874023438, route is: 0->6->15->7->9->1->3->2->11->5->12->8->13->14->10->4->0
ans is: 588.3677375976563, route is: 0->13->7->4->3->1->15->10->5->12->11->2->8->9->6->14->0
ans is: 577.4558905273437, route is: 0->6->13->7->15->8->11->4->12->2->1->9->3->5->14->10->0
ans is: 480.27078686523436, route is: 0->15->8->3->11->12->14->10->4->13->2->9->7->5->1->6->0


Test with number of 17 begin!
ans is

KeyboardInterrupt: 

In [ ]:
from openpyxl import load_workbook

# 载入已存在的xlsx文件
wb = load_workbook('result.xlsx')

# 选择一个工作表
ws = wb["Accurate"]

# 写入数据到第A列（你可以更改列名来选择不同的列）
ws[f'D{1}'] = "h=0"
for i, item in enumerate(result, start=2):  # start=1 表示从第一行开始
    ws[f'D{i}'] = np.mean(item)
# 保存文件
wb.save('result.xlsx')